In [1]:
import pyarabic.araby as araby
import pyarabic.number as number
import string
import re
import pickle
from collections import Counter
import torch
from torch.utils.data.dataset import Dataset, TensorDataset
from torch.utils.data.dataloader import DataLoader

In [ ]:
# letter_to_id = {}
# id_to_letter = {}
# word_to_id = {}
# id_to_word = {}
# diacritic_to_id = {}
# id_to_diacritic = {}
# word_count = Counter()

# id_letter = 0
# id_word = 0
# id_diacritic = 1

# diacritic_to_id["space"] = 0
# id_to_diacritic[0] = "space"

with open('Dataset_with_shaddah/test_cleaned_withshadda_215.txt','r',encoding='utf-8') as file:
    for line in file:
        letters, diacritics =araby.separate(line)
        index = 0
        for index,letter in enumerate(letters):
            if (letter == '\n') or (letter == '\u200f'):
                continue
                
            if letter == 'ّ':
                if diacritics[index] != 'ـ' :
                    shaddad_with_diac = letter+diacritics[index]
                    if shaddad_with_diac not in diacritic_to_id:
                        diacritic_to_id[shaddad_with_diac] = id_diacritic
                        id_to_diacritic[id_diacritic] = shaddad_with_diac
                        id_diacritic += 1
                else:
                    if letter not in diacritic_to_id:
                        diacritic_to_id[letter] = id_diacritic
                        id_to_diacritic[id_diacritic] = letter
                        id_diacritic += 1
            else:
                if letter not in letter_to_id:
                    letter_to_id[letter] = id_letter
                    id_to_letter[id_letter] = letter
                    id_letter += 1
                    
                diacritic = diacritics[index]
                if diacritic not in diacritic_to_id:
                    diacritic_to_id[diacritic] = id_diacritic
                    id_to_diacritic[id_diacritic] = diacritic
                    id_diacritic += 1
                    
        words = araby.tokenize(line)
        for word in words:
            if word == '\n':
                continue
            word_count[word] += 1
            
            if word not in word_to_id:
                word_to_id[word] = id_word
                id_to_word[id_word] = word
                id_word +=1
                
     
        

In [ ]:
with open('Dataset_with_shaddah/letter_to_id.pickle', 'wb') as file:
    pickle.dump(letter_to_id, file)
    
with open('Dataset_with_shaddah/id_to_letter.pickle', 'wb') as file:
    pickle.dump(id_to_letter, file)

In [ ]:
with open('Dataset_with_shaddah/word_to_id.pickle', 'wb') as file:
    pickle.dump(word_to_id, file)
    
with open('Dataset_with_shaddah/id_to_word.pickle', 'wb') as file:
    pickle.dump(id_to_word, file)

In [ ]:
with open('Dataset_with_shaddah/diacritic_to_id.pickle', 'wb') as file:
    pickle.dump(diacritic_to_id, file)
    
with open('Dataset_with_shaddah/id_to_diacritic.pickle', 'wb') as file:
    pickle.dump(id_to_diacritic, file)

In [ ]:
with open('Dataset_with_shaddah/word_count.pickle', 'wb') as file:
    pickle.dump(word_count, file)

In [ ]:
with open('Dataset_with_shaddah/id_to_word.pickle', 'rb') as file:
    with_shaddah = pickle.load(file)
len(with_shaddah)

In [ ]:
with open('Dataset/id_to_word.pickle', 'rb') as file:
    no_shaddah = pickle.load(file)
len(no_shaddah)

In [2]:
class DiacriticDatasetShaddah(Dataset):
    def __init__(self,dataset_path,letter_to_id_path,id_to_letter_path,diacritic_to_id_path,id_to_diacritic_path,word_to_id_path,id_to_word_path):
        
        self.file = dataset_path
        
        letter_to_id_file= open(letter_to_id_path, 'rb')
        self.letter_to_id = pickle.load(letter_to_id_file)
        letter_to_id_file.close()
        
        id_to_letter_file = open(id_to_letter_path, 'rb')
        self.id_to_letter = pickle.load(id_to_letter_file)
        id_to_letter_file.close()
        
        diacritic_to_id_file= open(diacritic_to_id_path, 'rb')
        self.diacritic_to_id = pickle.load(diacritic_to_id_file)
        diacritic_to_id_file.close()
        
        id_to_diacritic_file = open(id_to_diacritic_path, 'rb')
        self.id_to_diacritic = pickle.load(id_to_diacritic_file)
        id_to_diacritic_file.close()
        
        word_to_id_file= open(word_to_id_path, 'rb')
        self.word_to_id = pickle.load(word_to_id_file)
        word_to_id_file.close()
        
        id_to_word_file = open(id_to_word_path, 'rb')
        self.id_to_word = pickle.load(id_to_word_file)
        id_to_word_file.close()
        
        self.data = self.prepare_dataset()
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
        return self.data[index]
    
    def prepare_dataset(self):
        data = {}
        counter = 0
        with open(self.file,encoding='utf-8') as file:
            for line in file:
                letter_ids = []
                diacritic_ids = []
                word_ids = []
                letters, diacritics =araby.separate(line)
                letters = letters[0:-1]
                words = araby.tokenize(line)[0:-1]
                diacritics = diacritics[0:-1]
                index = 0
                for letter in letters:
                    if (letter == '\n') or (letter == '\u200f'):
                        continue
                    if (letter =='ّ'):
                        if diacritics[index] != 'ـ':
                            diacritic_ids[-1] = self.diacritic_to_id[letter+diacritics[index]]
                            
                        else:
                            diacritic_ids[-1] = self.diacritic_to_id[letter]
                        
                            
                    else:
                        letter_ids.append(self.letter_to_id[letter])
                        if letter == " ":
                            diacritic_ids.append(self.diacritic_to_id['space'])
                            
                        else:
                            diacritic_ids.append(self.diacritic_to_id[diacritics[index]])
                            
                    index += 1
                    
                for word in words:
                    word_ids.append(self.word_to_id[word])

                instance = (torch.tensor(letter_ids,dtype=torch.long,requires_grad=False),
                           torch.tensor(diacritic_ids,dtype=torch.long,requires_grad=False),
                           torch.tensor(word_ids,dtype=torch.long,requires_grad=False))
                data[counter] = instance
                counter += 1
        return data

In [3]:
train = DiacriticDatasetShaddah('Dataset_with_shaddah/train_cleaned_withshadda_215.txt','Dataset_with_shaddah/letter_to_id.pickle','Dataset_with_shaddah/id_to_letter.pickle','Dataset_with_shaddah/diacritic_to_id.pickle','Dataset_with_shaddah/id_to_diacritic.pickle','Dataset_with_shaddah/word_to_id.pickle','Dataset_with_shaddah/id_to_word.pickle')

In [6]:
train_dataloader = DataLoader(train)

In [12]:
for batch_idx, input_data in enumerate(train_dataloader):
    letters = input_data[0]
    labels = input_data[1]
    if letters.size(1) != labels.size(1):
        print("Error")